In [1]:
import numpy as np

In [59]:
def normalize(M):
    """
    Subfunction for T. It normalizes the matrix given as input.
    
    INPUT:
        - M = A matrix M.
        
    OUTPUT:
        - M0 = The matrix M normalized, with rows that add to 1.
    """
    
    M=np.array(M)
    s=M.sum(axis=1)
    return np.divide(M,s[:,np.newaxis])

In [75]:
def stationary_sol(T):
    """
    Given a random matrix T, returns a stationary sol a, s.t. aT=a
    
    INPUT:
        - T = Some square matrix.
        
    OUTPUT:
        - the stationary vector of the matrix.
    """
    T0=normalize(T)
    eig=np.linalg.eig(T0)
    for i in range(len(eig)):
        if eig[0][i]==1:
            return eig[1][i]
    return "No eigenvalue 1"

In [48]:
def T_non_reversible(C):
    """
    Function to get the transition matrix from the count matrix. It simply normalizes the count matrix.
    Is easy, and useful for very large amount of data.
    
    INPUT:
        - C = Count matrix.
    
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    return normalize(C)

In [ ]:
def obtain_irred(T):
    """
    Subfunction for T_reversible. It gets the biggest component of the matrix that we are given.
    
    INPUT:
        - T = The probability transition matrix of the markov model.
    
    OUTPUT:
        - C = The biggest connected component of the matrix.
    
    """

In [71]:
def T_reversible(C,max_iterations=100,error=0.1):
    """
    Function to get the transition matrix from the count matrix. It gives a matrix that is reversible.
    That is, the markov model obtained is reversible (it satisfies the detailed balance equations).
    Detailed balance implies that, around any closed cycle of states, there is no net flow of probability. 
    For example, it implies that, for all a, b and c,
    T( a , b ) T( b , c ) T( c , a ) = T( a , c ) T( c , b ) T( b , a ).
    
    INPUT:
        - C = Count matrix constructed with lag tau.
        - max_iterations = maximum number of iterations we allow.
        - error = error that we consider to understand that the iteration has converged.
        
    OUTPUT:
        - P = The probability transition matrix of the markov model.
    """
    
    C_i = C.sum(axis=1) #array of the sums of the rows of C
    C_j = C.sum(axis=0) #array of the sums of the columns of C
    
    P = T_non_reversible(C)
    P = obtain_irred(P)
    pi = stationary_sol(P)
    
    print(pi)
    print(P)
    
    P=np.multiply(P,pi)
    X_0= P #initial state
    
    it=0
    Er=0.2 #TO BE CHANGED
    
    while (Er > error) and (it< max_iterations):
        Xi_0= X_0.sum(axis=1) #array of the sums of the rows of X_0
        Xj_0= X_0.sum(axis=0) #array of the sums of the rows of X_0
        
        X_1= C + np.matrix.transpose(C)
        X_1=np.divide(X_1,((C_i)/(Xi_0) + (C_j)/(Xj_0)))
        
        X_0 = X_1
        it+=1
        
    P = normalize(X_1)
    return P

In [50]:
M1=np.array([[0.5,0.5],[0.1,0.9]])
M1

array([[ 0.5,  0.5],
       [ 0.1,  0.9]])

In [51]:
C=np.array([[ 0, 1, 0],[0,1,1],[ 1, 0,0]])
C

array([[0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [52]:
C0=np.array(C)
C0

array([[0, 1, 0],
       [0, 1, 1],
       [1, 0, 0]])

In [53]:
s=C0.sum(axis=1)
s

array([1, 2, 1])

In [54]:
s[:,np.newaxis]

array([[1],
       [2],
       [1]])

In [55]:
ss=np.array([[1,1,2],[1,2,2],[2,2,2]])
ss

array([[1, 1, 2],
       [1, 2, 2],
       [2, 2, 2]])

In [56]:
C0/=ss

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'l') according to the casting rule ''same_kind''

In [42]:
C0/=s[:,np.newaxis]

TypeError: ufunc 'true_divide' output (typecode 'd') could not be coerced to provided output parameter (typecode 'l') according to the casting rule ''same_kind''

In [60]:
normalize(count_matrix)

array([[ 0. ,  1. ,  0. ],
       [ 0. ,  0.5,  0.5],
       [ 1. ,  0. ,  0. ]])

In [61]:
T_non_rev(count_matrix)

array([[ 0. ,  1. ,  0. ],
       [ 0. ,  0.5,  0.5],
       [ 1. ,  0. ,  0. ]])

In [72]:
T_rev(count_matrix)

None
[[ 0.   1.   0. ]
 [ 0.   0.5  0.5]
 [ 1.   0.   0. ]]


TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [77]:
stationary_sol(T_non_rev(count_matrix))

'No eigenvalue 1'

In [78]:
eig=np.linalg.eig(count_matrix)
eig

(array([-0.23278562+0.79255199j, -0.23278562-0.79255199j,  1.46557123+0.j        ]),
 array([[-0.15883610+0.54078026j, -0.15883610-0.54078026j, -0.52606300+0.j        ],
        [-0.39162171-0.25177173j, -0.39162171+0.25177173j, -0.77098280+0.j        ],
        [ 0.68232780+0.j        ,  0.68232780-0.j        , -0.35894741+0.j        ]]))

In [76]:
T_non_rev(count_matrix)

array([[ 0. ,  1. ,  0. ],
       [ 0. ,  0.5,  0.5],
       [ 1. ,  0. ,  0. ]])

In [ ]:
stationary_sol()